In [1]:
#setup data
import pandas as pd
import numpy as np
import math
import random
from sklearn import preprocessing

ignore_features = ['25', '26', '8', '58', '20', '23', '29', '31', '32', '35']
toencode = {}
f = open("field_types.txt", 'r')
for line in f.readlines():
    line = line.strip().split(" ")
    if line[0] not in ignore_features:
        toencode[line[0]] = len(line) - 1
        

f1 = open("data.csv")
data_df = pd.read_csv(f1, sep=',', header = 'infer', low_memory=False)
data_df = data_df.drop(ignore_features, axis=1)
label_df = data_df[['label']]
data_df = data_df.drop('label', axis=1)
label = np.array(label_df)


f2 = open("quiz.csv")
quiz_df = pd.read_csv(f2, sep=',', header = 'infer', low_memory=False)
quiz_df = quiz_df.drop(ignore_features, axis=1)


In [2]:
#getting test and train
data_end = np.shape(data_df)[0]
df = data_df.append(quiz_df)
df_trans = pd.get_dummies(df, columns=toencode.keys(), dummy_na=False)
data = np.array(df_trans)

data, quiz = data[:data_end], data[data_end:]

In [3]:
#splitting to features and labels by training and test
def splitdata(data, label):
    n = data.shape[0]
    train_num = int(math.ceil(n*.8))
    indices = random.sample(xrange(n), n)

    training_idx, test_idx = indices[train_num:], indices[:train_num]
    training, test = data[training_idx,:], data[test_idx,:]
    lab, testlabel = label[training_idx,:], label[test_idx,:]
    
    return training, lab, test, testlabel

train, lab, test, testlabel = splitdata(data, label)

In [4]:
#evaluator
def eval(preds, testlabels):
    if len(preds) == len(testlabels):
        error = np.count_nonzero(preds != testlabels) / np.float(len(preds))
        return 1 - error

In [5]:
#unraveling
testlabelnum = np.ravel(testlabel)
labnum = np.ravel(lab)
#testtranslabelnum = np.ravel(testtrans)

In [ ]:
testtranslabelnum = np.ravel(testtrans)

In [6]:
# scalar weighting
scaler = preprocessing.StandardScaler().fit(train)
train = scaler.transform(train) 
test = scaler.transform(test) 
quiz = scaler.transform(quiz) 

In [ ]:
# csv writer
import csv
csvfile = "adaboost.csv"

with open(csvfile,"w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in yadaboostquiz:
        writer.writerow([val])

In [21]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

#logistic regression (0.87656)
ylogregtest = logreg.fit(train, lab).predict(test)
print eval(ylogregtest, testlabelnum)

0.878417266187


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
#decision tree 30 depth (0.890)
from sklearn.tree import DecisionTreeClassifier
dectree30 = DecisionTreeClassifier(max_depth=30)

ydectree30test = dectree30.fit(train, lab).predict(test)
print eval(ydectree30test, testlabelnum)

0.896255050754


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1000)
pca.fit(train,lab)
traintrans = pca.fit_transform(train,lab)
testtrans = pca.transform(test)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

#logistic regression (0.87656)
ylogregtest = logreg.fit(traintrans, lab).predict(testtrans)
print eval(ylogregtest, testtranslabelnum)

In [44]:
#decision tree 30 depth (0.890)
from sklearn.tree import DecisionTreeClassifier
dectree30 = DecisionTreeClassifier(max_depth=30)

ydectree30test = dectree30.fit(traintrans, lab).predict(testtrans)
print eval(ydectree30test, testtranslabelnum)

None


In [36]:
print eval(ylogregtest, testtranslabelnum)
print eval(ydectree30test, testtranslabelnum)

None
None


In [38]:
ylogregtest

array([-1, -1,  1, ..., -1,  1,  1])

In [7]:
#RandomForestClassifier (0.89812)
from sklearn.ensemble import RandomForestClassifier
randforest = RandomForestClassifier(max_depth=30)

yrandforest = randforest.fit(train, lab).predict(test)
print eval(yrandforest, testlabelnum)


/Users/radixold/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


NameError: name 'testtranslabelnum' is not defined

In [8]:
print eval(yrandforest, testlabelnum)

0.898127525377


In [10]:
#GaussianNB (0.7055)
from sklearn.naive_bayes import GaussianNB
gaussnb = GaussianNB()

ygauss = gaussnb.fit(train, lab).predict(test)
print eval(ygauss, testlabelnum)

0.705489307184


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
#LDA ()
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()

ylda = lda.fit(train, lab).predict(test)
print eval(ylda, testlabelnum)


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


TypeError: object of type 'LinearDiscriminantAnalysis' has no len()

In [14]:
#QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()

yqda = qda.fit(train, lab).predict(test)
print eval(yqda, testlabelnum)

/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:688: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


TypeError: object of type 'QuadraticDiscriminantAnalysis' has no len()

In [15]:
print eval(ylda, testlabelnum)
print eval(yqda, testlabelnum)

0.880358726717
0.745855918005


In [16]:
#SGD (0.8332)
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss="hinge", penalty="l2")

ysgd = sgd.fit(train, lab).predict(test)
print eval(ysgd, testlabelnum)

0.833261062383


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#KNN (0.8332)
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss="hinge", penalty="l2")

ysgd = sgd.fit(train, lab).predict(test)
print eval(ysgd, testlabelnum)

In [29]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=(.9 * (1 - .9)))
trainvt = vt.fit_transform(train)
testvt = vt.transform(test)

In [ ]:
testvt = vt.transform(test)

In [30]:
from sklearn.linear_model import LogisticRegression
logregvt = LogisticRegression()

#logistic regression (0.87656)
ylogregtestvt = logregvt.fit(trainvt, lab).predict(testvt)
print eval(ylogregtest, testlabelnum)

/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NameError: name 'ylogregtest' is not defined

In [ ]:
print eval(ylogregtestvt, testlabelnum)

In [27]:
ylogregtestvt.shape

(25367,)

In [28]:
testlabelnum.shape

(101470,)